In [1]:
#več o astropy -> https://docs.astropy.org/en/stable/index.html
import astropy.coordinates as coord
from astropy.time import Time
import astropy.units as u

import time
try:
    from tqdm.notebook import tqdm
except:
    !pip install tqdm
    from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from calendar import monthrange

In [2]:
def plot_vzhod(x1,x2,y):
    '''
    V polarnih koordinatah izriše
    lego sončnega vzhoda in zahoda glede na
    dolžino dneva
    x1 -> list: vsebuje podatke o azimutu sončnega vzhoda podanega v °
    x2 -> list: vsebuje podatke o azimutu sončnega zahoda podanega v °
    y -> list: vsebuje podatke o dolžini dneva v enotah ura
    '''
    fig = plt.figure()
    ax = plt.axes(polar=True)
    x1 =  np.array([2*np.pi/360*xx for xx in x1])
    x2 =  np.array([2*np.pi/360*xx for xx in x2])
    y = np.array(y)
    ax.plot(x1, y, "go")
    ax.plot(x2, y, "ro")
    ax.set_ylim(4,20)
    plt.show()
    
def convert_to_string(xx,ref):
    m=str(xx)
    m_len=len(m)
    m_levo=(ref-m_len)//2
    m_desno=ref-m_len-m_levo
    m1=' '.join('' for k in range(m_levo))
    m2=' '.join('' for k in range(m_desno))
    return m1+m+m2
    
def convert_time_to_float(t):
    '''
    čas je podan v spremenljivki t kot string, ki ima format
    hh:mm:ss -> ta zapis pretvorimo v realno število enota pa
    so ure t.split(:)->[hh,mm,ss]
    '''
    h=float(t.split(":")[0])
    m=float(t.split(":")[1])/60.0
    s=float(t.split(":")[2])/3600.0
    return h+m+s

def convert(t):
    '''
    tukaj pretvorimo string, ki odraža podatek o datumu in uri v
    datetime spremenljivko.
    '''
    return time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S'))

def convert_deg(d):
    '''
    d je string, ki vsebuje podatke o kotu in je podan v obliki
    41d52m31s
    s spodnjo kodo tako zapisan podatek pretvorimo v realno število,
    ki odraža stopinje.
    '''
    deg=d.split("d")[0]
    m=d.split("d")[1].split("m")[0]
    s=d.split("m")[1].split("s")[0]
    deg=float(deg)+float(m)/60+float(s)/3600
    return deg

## Lokacija

In [3]:
#knjižica omogoča dostop do lon in lat lokacij na različne načine

observatoriji=coord.EarthLocation.get_site_names()

for o in tqdm(observatoriji[0:10],total=len(observatoriji),desc="Observatoriji"):
    try:
        loc=coord.EarthLocation.of_address(o)
        print(o,loc.lat.degree,loc.lon.degree)
    except:
        pass

Observatoriji:   0%|          | 0/322 [00:00<?, ?it/s]

ALMA 48.54888699999999 -71.65145899999999
AO 47.593970000000006 14.124559999999999
ARCA 56.0909916 49.8771828
ATA -21.207991999999997 -50.43902250000002
ATST 34.75162634999999 135.39237939999998
Allen Telescope Array 40.81713210000001 -121.46904400000003
Anderson Mesa 35.0552961 -111.44042919999998
Anglo-Australian Observatory -31.275451999999994 149.067221
Apache Point 35.795316499999984 -104.17914339999996


In [5]:
loc=coord.EarthLocation.of_address("Koroška cesta 160, Maribor, Slovenija")
print(loc)
lat=loc.lat.degree
lon=loc.lon.degree
print(lat,lon)

(4230704.25652027, 1183154.23497847, 4608555.13683359) m
46.563743900000006 15.6241238


In [46]:
df_1min={}
df_1min["Datum"]=[]
df_1min["Mesec"]=[]
df_1min["Dan"]=[]
df_1min["Ura"]=[]
df_1min["Minuta"]=[]
df_1min["Az"]=[]
df_1min["Alt"]=[]

df_dnevno={}
df_dnevno["Datum"]=[]
df_dnevno["Mesec"]=[]
df_dnevno["Dan"]=[]
df_dnevno["Sončni vzhod lokacija"]=[]
df_dnevno["Sončni vzhod ura"]=[]
df_dnevno["Sončni zahod lokacija"]=[]
df_dnevno["Sončni zahod ura"]=[]
df_dnevno["Dolžina dneva"]=[]
df_dnevno["Alt maks"]=[]

In [50]:
leto=2023

dmesec=1
ddan=1

start_h=0
end_h=24
dh=1
for mesec in tqdm(range(1,13,dmesec),desc="Mesec"):
    stevilo_dni=monthrange(leto, mesec)[1]
    for dan in tqdm(range(1,stevilo_dni+1,ddan),desc="Dan"):
        
        vzhod=False
        alt_list=[]
        
        vzhod_loc=None
        vzhod_t=None
        
        zahod_loc=None
        zahod_t=None
        for h in range(start_h,end_h,dh):
            for m in range(0,60,1):
                t=f"{leto:04d}-{mesec:02d}-{dan:02d}T{h:02d}:{m:02d}:00"
                t="%04d-%02d-%02dT%02d:%02d:00"%(leto,mesec,dan,h,m)
                altaz = coord.AltAz(location=loc, obstime=t)
                sun = coord.get_sun(Time(t, format='isot', scale='utc'))
                y=sun.transform_to(altaz).alt.degree
                y2=sun.transform_to(altaz).az.degree
                df_1min["Datum"].append(t)
                df_1min["Mesec"].append(mesec)
                df_1min["Dan"].append(dan)
                df_1min["Ura"].append(h)
                df_1min["Minuta"].append(m)
                df_1min["Az"].append(y2)
                df_1min["Alt"].append(y)
                
                alt_list.append(y)
                
                if not vzhod and len(df_1min["Alt"])>1:
                    if df_1min["Alt"][-2]*df_1min["Alt"][-1]<0:
                        vzhod=True
                        vzhod_loc=y2
                        vzhod_t=h+m/60.0

                if vzhod and df_1min["Alt"][-2]>0 and df_1min["Alt"][-1]<0:
                    
                    zahod_loc=y2
                    zahod_t=h+m/60.0
        dt=zahod_t-vzhod_t
        
        df_dnevno["Datum"].append(t)
        df_dnevno["Mesec"].append(mesec)
        df_dnevno["Dan"].append(dan)
        df_dnevno["Sončni vzhod lokacija"].append(vzhod_loc)
        df_dnevno["Sončni vzhod ura"].append(vzhod_t)
        df_dnevno["Sončni zahod lokacija"].append(zahod_loc)
        df_dnevno["Sončni zahod ura"].append(zahod_t)
        df_dnevno["Dolžina dneva"].append(dt)
        df_dnevno["Alt maks"].append(max(alt_list))
        print(dan,mesec,leto,"%.2f"%dt,"%.2f"%vzhod_t,"%.2f"%zahod_t,"%.2f"%max(alt_list))

Mesec:   0%|          | 0/12 [00:00<?, ?it/s]

Dan:   0%|          | 0/31 [00:00<?, ?it/s]

1 1 2023 15.25 0.00 15.25 20.43
2 1 2023 8.47 6.80 15.27 20.52
3 1 2023 8.48 6.80 15.28 20.61
4 1 2023 8.50 6.80 15.30 20.71
5 1 2023 8.52 6.80 15.32 20.82
6 1 2023 8.55 6.78 15.33 20.94
7 1 2023 8.57 6.78 15.35 21.06


KeyboardInterrupt: 

In [51]:
import pandas as pd
df_dnevno=pd.DataFrame.from_dict(df_dnevno)
df_dnevno.to_csv("solar_daily.csv")

df_1min=pd.DataFrame.from_dict(df_1min)
df_1min.to_csv("solar_minute.csv")

In [52]:
df_dnevno

,Datum,Mesec,Dan,Sončni vzhod lokacija,Sončni vzhod ura,Sončni zahod lokacija,Sončni zahod ura,Dolžina dneva,Alt maks
0,2023-01-01T19:59:00,1,1,124.756877,6.800000,235.474433,15.250000,8.450000,20.431124
1,2023-01-01T23:59:00,1,1,124.756877,6.800000,235.474433,15.250000,8.450000,20.431124
2,2023-01-01T23:59:00,1,1,31.997377,0.000000,235.474433,15.250000,15.250000,20.431124
3,2023-01-02T23:59:00,1,2,124.619636,6.800000,235.624840,15.266667,8.466667,20.517080
4,2023-01-03T23:59:00,1,3,124.478743,6.800000,235.781097,15.283333,8.483333,20.610795
5,2023-01-04T23:59:00,1,4,124.334295,6.800000,235.943269,15.300000,8.500000,20.711927
6,2023-01-05T23:59:00,1,5,124.186392,6.800000,236.111419,15.316667,8.516667,20.820664
7,2023-01-06T23:59:00,1,6,123.853576,6.783333,236.285603,15.333333,8.550000,20.936771
8,2023-01-07T23:59:00,1,7,123.698998,6.783333,236.465876,15.350000,8.566667,21.060300


In [ ]:
# Graf ki prikazuje
1) Kako se spreminja Dolžina dneva glede na zaporeden dan v letu
2) Kako se spreminja Lega sonca ob vzhodu glede na zaporeden dan v letu
3) Kako se spreminja Lega sonca ob zahodu glede na zaporeden dan v letu
4) Kako se spreminja najvija lega sonca ob glede na zaporeden dan v letu

Možnosti za eksperimentiranje:
Primerjaj grafe za lokacijo blizu ekvatorje in za lokacijo blizu pola